In [5]:
# %pip install datasets
# %pip install nltk
# %pip install tensorflow
# %pip install gensim

# Part 0. Data Prepraration

In [1]:
from datasets import load_dataset
dataset = load_dataset("rotten_tomatoes")
train_dataset = dataset ['train']
validation_dataset = dataset ['validation']
test_dataset = dataset ['test']

c:\Users\Alan Wong\Desktop\SC4002\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# removing stopwords

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
def remove_stopwords(example):
    example['text'] = ' '.join(
        [word for word in example['text'].split() if word.lower() not in stop_words]
    )
    return example

# Apply the function to each split
train_dataset = train_dataset.map(remove_stopwords)
validation_dataset = validation_dataset.map(remove_stopwords)
test_dataset = test_dataset.map(remove_stopwords)
print(train_dataset[0]['text'])  # Should display text without stopwords

[nltk_data] Downloading package stopwords to C:\Users\Alan
[nltk_data]     Wong\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
Map: 100%|██████████| 1066/1066 [00:00<00:00, 15952.08 examples/s]

rock destined 21st century's new " conan " he's going make splash even greater arnold schwarzenegger , jean-claud van damme steven segal .


# Part 1. Preparing Word Embeddings

#### (a) What is the size of the vocabulary formed from your training data?

In [5]:
import nltk
import numpy as np
nltk.download('punkt')
# In case nltk.download('punkt') does not work.
# nltk.download('punkt_tab')
vocab = set()
for text in train_dataset['text']:
    ls = nltk.word_tokenize(text)
    for word in ls:
        if word.isalpha(): vocab.add(word)
#Size of the vocabulary: 15812
#New Size of the vocabulary: 15725

[nltk_data] Downloading package punkt to C:\Users\Alan
[nltk_data]     Wong\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
print("Size of the vocabulary:", len(vocab))

Size of the vocabulary: 15725


#### (b) We use OOV (out-of-vocabulary) to refer to those words appeared in the training data but not in the Word2vec (or Glove) dictionary. How many OOV words exist in your training data?

In [7]:
import gensim.downloader as api
for key in api.info()['models'].keys():
    print(key)

fasttext-wiki-news-subwords-300
conceptnet-numberbatch-17-06-300
word2vec-ruscorpora-300
word2vec-google-news-300
glove-wiki-gigaword-50
glove-wiki-gigaword-100
glove-wiki-gigaword-200
glove-wiki-gigaword-300
glove-twitter-25
glove-twitter-50
glove-twitter-100
glove-twitter-200
__testing_word2vec-matrix-synopsis


In [8]:
model = api.load("glove-wiki-gigaword-100")

In [9]:
oov_words = set()
for word in vocab:
    if word not in model:
        oov_words.add(word)

print("Number of OOV words:", len(oov_words))

Number of OOV words: 550


#### (c) The existence of the OOV words is one of the well-known limitations of Word2vec (or Glove). Without using any transformer-based language models (e.g., BERT, GPT, T5), what do you think is the best strategy to mitigate such limitation? Implement your solution in your source code. Show the corresponding code snippet.

In [10]:
# Group any words that are not in the model into a single token
def wordtovec(word):
    if word in model:
        return model[word]
    else:
        return np.zeros(model.vector_size)

# Part 2. Model Training & Evaluation - RNN

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import random

In [12]:
vocab_size = len(model.index_to_key) + 1
embedding_dim = model.vector_size
word_index = {word: index+1 for index, word in enumerate(model.index_to_key)} # index 0 is reserved for padding
embedding_matrix = np.zeros((vocab_size, embedding_dim))

In [13]:
for word, idx in word_index.items():
    if word in model:
        embedding_matrix[idx] = model[word]

In [14]:
def tokenize(text, word_index):
    ls = nltk.word_tokenize(text)
    return [word_index[word] for word in ls if word in word_index]

X_train = [tokenize(text, word_index) for text in train_dataset['text']]
X_val = [tokenize(text, word_index) for text in validation_dataset['text']]
X_test = [tokenize(text, word_index) for text in test_dataset['text']]
max_length = max(len(seq) for seq in X_train)

In [15]:
X_train = pad_sequences(X_train, maxlen=max_length, padding='post', truncating='post')
X_val = pad_sequences(X_val, maxlen=max_length, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=max_length, padding='post', truncating='post')

In [16]:
y_train = np.array(train_dataset['label'])
y_val = np.array(validation_dataset['label'])
y_test = np.array(test_dataset['label'])

Model Training - Grid Search

In [17]:
from tensorflow.keras.callbacks import Callback
best_accuracy = {}
class CustomCallback(Callback):
    accuracy = 0
    cur_key = ""
    epochs = 0
    optimizer = ""
    batch_size = 0
    best_model = None
    lr = 0
    def on_train_begin(self, logs=None):
        self.accuracy = 0

    def on_train_end(self, logs=None):
        global best_accuracy
        if self.accuracy > best_accuracy.get("accuracy", 0):
            best_accuracy = {
                "accuracy": self.accuracy,
                "epoch": self.epochs,
                "optimizer": self.optimizer,
                "batch_size": self.batch_size,
                "lr": self.lr
            }
            print("Saved best accuracy for current run:", self.accuracy, "at epoch", self.epochs)
            self.best_model.save(filepath="best_model.keras")
        print("Run completed on:")
        print(self.cur_key)
        print("Best accuracy for current run:", self.accuracy, "at epoch", self.epochs)
        print("Training ended")
    
    def on_epoch_end(self, epoch, logs=None):
        val_accuracy = logs['val_accuracy']
        if val_accuracy > self.accuracy:
            self.accuracy = val_accuracy
            self.epochs = epoch
            self.best_model = self.model

    def set_key(self, optimizer, batch_size, lr):
        self.optimizer = optimizer
        self.batch_size = batch_size
        self.lr = lr
        self.cur_key = f"optimizer: {optimizer}, batch_size: {batch_size}, lr: {lr}"

In [18]:
def train_model(optimizer, epochs, batch_size, lr):
    tf.random.set_seed(0)
    np.random.seed(0)
    random.seed(0)
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    )
    custom_callback = CustomCallback()
    custom_callback.set_key(optimizer, batch_size, lr)
    model = Sequential([
        Embedding(input_dim=vocab_size,
                  output_dim=embedding_dim,
                  weights=[embedding_matrix],
                  trainable=False),  # Embedding layer is frozen
        SimpleRNN(16, return_sequences=False),
        Dense(1, activation='sigmoid')
    ])
    if optimizer == 'adam': optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'sgd': optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    elif optimizer == 'rmsprop': optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    else: optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[custom_callback, early_stopping]
    )
    return model, history

In [19]:
for batch_size in [16, 32, 64, 128]:
    for lr in [0.005, 0.01, 0.05, 0.1]:
        for optimizer in ['adam', 'sgd', 'rmsprop', 'adagrad']:
            train_model(optimizer, 100, batch_size, lr)

Epoch 1/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.5137 - loss: 0.6954 - val_accuracy: 0.5000 - val_loss: 0.6940
Epoch 2/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5042 - loss: 0.6939 - val_accuracy: 0.5000 - val_loss: 0.6938
Epoch 3/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.4946 - loss: 0.6937 - val_accuracy: 0.5000 - val_loss: 0.6934
Epoch 4/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.5003 - loss: 0.6935 - val_accuracy: 0.5000 - val_loss: 0.6935
Epoch 5/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.4957 - loss: 0.6936 - val_accuracy: 0.5000 - val_loss: 0.6935
Epoch 6/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.4964 - loss: 0.6935 - val_accuracy: 0.5000 - val_loss: 0.6934
Saved best accuracy for current run: 0.5 at epoch 0
Run completed on:
optimizer: adam, batch_size: 16, lr: 0.005
Best accuracy for current run: 0.5 at epoch 0
Training ended
Epoch 1/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step

In [20]:
best_accuracy

{'accuracy': 0.696060061454773,
 'epoch': 77,
 'optimizer': 'adagrad',
 'batch_size': 128,
 'lr': 0.005}

Best model is trained with Optimizer: adagrad, Epoch: 35, Batch_size: 64, Learning_rate: 0.01

In [21]:
model, history = train_model("adagrad", 40, 64, 0.01)

Epoch 1/40
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5036 - loss: 0.6959 - val_accuracy: 0.5338 - val_loss: 0.6941
Epoch 2/40
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5619 - loss: 0.6842 - val_accuracy: 0.5694 - val_loss: 0.6839
Epoch 3/40
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5959 - loss: 0.6715 - val_accuracy: 0.6088 - val_loss: 0.6651
Epoch 4/40
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6179 - loss: 0.6607 - val_accuracy: 0.6351 - val_loss: 0.6517
Epoch 5/40
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6293 - loss: 0.6513 - val_accuracy: 0.6313 - val_loss: 0.6430
Epoch 6/40
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6421 - loss: 0.6423 - val_accuracy: 0.6332 - val_loss: 0.6367
Epoch 7/40
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6500 - loss: 0.6339 - val_accuracy: 0.6388 - val_loss: 0.6310
Epoch 8/40
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6603 - loss: 0.6264 - val_accuracy: 0.

In [22]:
best_model = tf.keras.models.load_model("best_model.keras")
accuracy = best_model.evaluate(X_val, y_val)
print("Test accuracy:", accuracy[1])

34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6564 - loss: 0.6093 
Test accuracy: 0.690431535243988


In [23]:
best_model.get_compile_config()

{'optimizer': {'module': 'keras.optimizers',
  'class_name': 'Adagrad',
  'config': {'name': 'adagrad',
   'learning_rate': 0.004999999888241291,
   'weight_decay': None,
   'clipnorm': None,
   'global_clipnorm': None,
   'clipvalue': None,
   'use_ema': False,
   'ema_momentum': 0.99,
   'ema_overwrite_frequency': None,
   'loss_scale_factor': None,
   'gradient_accumulation_steps': None,
   'initial_accumulator_value': 0.1,
   'epsilon': 1e-07},
  'registered_name': None},
 'loss': 'binary_crossentropy',
 'loss_weights': None,
 'metrics': ['accuracy'],
 'weighted_metrics': None,
 'run_eagerly': False,
 'steps_per_execution': 1,
 'jit_compile': False}

#### Mean pooling

In [24]:
from tensorflow.keras.layers import AveragePooling1D, GlobalAveragePooling1D
def train_model(optimizer, epochs, batch_size, lr):
    tf.random.set_seed(0)
    np.random.seed(0)
    random.seed(0)
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    )
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath="model_mean.keras", 
        monitor='val_accuracy',            
        save_best_only=True,           
        mode='max',                 
        save_weights_only=False,       
        verbose=1
    )
    model = Sequential([
        Embedding(input_dim=vocab_size,
                  output_dim=embedding_dim,
                  weights=[embedding_matrix],
                  trainable=False),  # Embedding layer is frozen
        SimpleRNN(16, return_sequences=True),
        GlobalAveragePooling1D(),
        Dense(1, activation='sigmoid')
    ])
    if optimizer == 'adam': optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'sgd': optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    elif optimizer == 'rmsprop': optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    else: optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[checkpoint_callback, early_stopping]
    )
    return model, history

In [25]:
model, history = train_model("adagrad", 100, 64, 0.01)

Epoch 1/100
131/134 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4578 - loss: 0.7000
Epoch 1: val_accuracy improved from -inf to 0.49531, saving model to model_mean.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.4584 - loss: 0.6999 - val_accuracy: 0.4953 - val_loss: 0.6946
Epoch 2/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4925 - loss: 0.6950
Epoch 2: val_accuracy improved from 0.49531 to 0.51313, saving model to model_mean.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.4926 - loss: 0.6950 - val_accuracy: 0.5131 - val_loss: 0.6928
Epoch 3/100
131/134 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5066 - loss: 0.6932
Epoch 3: val_accuracy improved from 0.51313 to 0.52533, saving model to model_mean.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5069 - loss: 0.6932 - val_accuracy: 0.5253 - val_loss: 0.6914
Epoch 4/100
129/134 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5203 - loss: 0.6917
Epoch 4: val_accuracy improved from 

In [26]:
best_model = tf.keras.models.load_model("model_mean.keras")
accuracy = best_model.evaluate(X_test, y_test)
print("Test accuracy:", accuracy[1])

34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7345 - loss: 0.5605 
Test accuracy: 0.6848030090332031


#### Max pooling

In [27]:
from tensorflow.keras.layers import MaxPooling1D, GlobalMaxPooling1D
def train_model(optimizer, epochs, batch_size, lr):
    tf.random.set_seed(0)
    np.random.seed(0)
    random.seed(0)
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    )
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath="model_max.keras", 
        monitor='val_accuracy',            
        save_best_only=True,           
        mode='max',                 
        save_weights_only=False,       
        verbose=1
    )
    model = Sequential([
        Embedding(input_dim=vocab_size,
                  output_dim=embedding_dim,
                  weights=[embedding_matrix],
                  trainable=False),  # Embedding layer is frozen
        SimpleRNN(16, return_sequences=True),
        GlobalMaxPooling1D(),
        Dense(1, activation='sigmoid')
    ])
    if optimizer == 'adam': optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'sgd': optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    elif optimizer == 'rmsprop': optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    else: optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[checkpoint_callback, early_stopping]
    )
    return model, history

In [28]:
model, history = train_model("adagrad", 100, 64, 0.01)

Epoch 1/100
126/134 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5095 - loss: 0.7062
Epoch 1: val_accuracy improved from -inf to 0.52439, saving model to model_max.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5096 - loss: 0.7057 - val_accuracy: 0.5244 - val_loss: 0.6920
Epoch 2/100
127/134 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5311 - loss: 0.6902
Epoch 2: val_accuracy improved from 0.52439 to 0.54034, saving model to model_max.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5311 - loss: 0.6902 - val_accuracy: 0.5403 - val_loss: 0.6882
Epoch 3/100
128/134 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5481 - loss: 0.6856
Epoch 3: val_accuracy improved from 0.54034 to 0.54597, saving model to model_max.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5482 - loss: 0.6856 - val_accuracy: 0.5460 - val_loss: 0.6838
Epoch 4/100
126/134 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5686 - loss: 0.6804
Epoch 4: val_accuracy improved from 0.54

In [29]:
best_model = tf.keras.models.load_model("model_max.keras")
accuracy = best_model.evaluate(X_test, y_test)
print("Test accuracy:", accuracy[1])

34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6999 - loss: 0.5836   
Test accuracy: 0.7298311591148376


#### Dense layer

In [30]:
from tensorflow.keras.layers import Flatten
def train_model(optimizer, epochs, batch_size, lr):
    tf.random.set_seed(0)
    np.random.seed(0)
    random.seed(0)
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    )
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath="model_dense.keras", 
        monitor='val_accuracy',            
        save_best_only=True,           
        mode='max',                 
        save_weights_only=False,       
        verbose=1
    )
    model = Sequential([
        Embedding(input_dim=vocab_size,
                  output_dim=embedding_dim,
                  weights=[embedding_matrix],
                  trainable=True),  # Embedding layer is frozen
        SimpleRNN(16, return_sequences=True),
        Flatten(),
        Dense(62, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    if optimizer == 'adam': optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'sgd': optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    elif optimizer == 'rmsprop': optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    else: optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[checkpoint_callback, early_stopping]
    )
    return model, history

In [31]:
model, history = train_model("adagrad", 100, 64, 0.01)

Epoch 1/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - accuracy: 0.5417 - loss: 0.6986
Epoch 1: val_accuracy improved from -inf to 0.58068, saving model to model_dense.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 17s 114ms/step - accuracy: 0.5418 - loss: 0.6985 - val_accuracy: 0.5807 - val_loss: 0.6771
Epoch 2/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - accuracy: 0.6285 - loss: 0.6447
Epoch 2: val_accuracy improved from 0.58068 to 0.62570, saving model to model_dense.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 15s 108ms/step - accuracy: 0.6286 - loss: 0.6447 - val_accuracy: 0.6257 - val_loss: 0.6579
Epoch 3/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - accuracy: 0.6698 - loss: 0.6096
Epoch 3: val_accuracy improved from 0.62570 to 0.65666, saving model to model_dense.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 15s 108ms/step - accuracy: 0.6698 - loss: 0.6095 - val_accuracy: 0.6567 - val_loss: 0.6324
Epoch 4/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - accuracy: 0.6942 - loss: 0.5847
Epoch 4: val_accura

In [32]:
best_model = tf.keras.models.load_model("model_dense.keras")
accuracy = best_model.evaluate(X_test, y_test)
print("Test accuracy:", accuracy[1])

34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6725 - loss: 0.6366 
Test accuracy: 0.7204502820968628


# Part 3. Enhancement


#### 1. Instead of keeping the word embeddings fixed, now update the word embeddings (the same way as model parameters) during the training process.


In [30]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

tf.random.set_seed(0)
np.random.seed(0)
random.seed(0)
model = Sequential([
    Embedding(input_dim=vocab_size,
              output_dim=embedding_dim,
              weights=[embedding_matrix],
              trainable=True),
    SimpleRNN(16, return_sequences=False),
    Dense(1, activation='sigmoid')
])
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01) #Static learning rate
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=32,
    callbacks=[early_stopping]
)

Epoch 1/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 68s 247ms/step - accuracy: 0.5068 - loss: 0.6994 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 2/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 67s 250ms/step - accuracy: 0.5133 - loss: 0.6952 - val_accuracy: 0.5000 - val_loss: 0.6955
Epoch 3/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 66s 248ms/step - accuracy: 0.5070 - loss: 0.6956 - val_accuracy: 0.5000 - val_loss: 0.6951
Epoch 4/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 66s 246ms/step - accuracy: 0.5063 - loss: 0.6955 - val_accuracy: 0.5000 - val_loss: 0.6949


#### 2. As discussed in Question 1(c), apply your solution in mitigating the influence of OOV words and train your model again.
##### (Run cells 6 and 7 first)

In [34]:
vocab_size = len(model.index_to_key) + 2 # 0 is reserved for padding, 1 is reserved for OOV
embedding_dim = model.vector_size
word_index = {word: index+2 for index, word in enumerate(model.index_to_key)} # index 0 is reserved for padding
embedding_matrix = np.zeros((vocab_size, embedding_dim))

In [35]:
for word, idx in word_index.items():
    if word in model:
        embedding_matrix[idx] = model[word]

In [36]:
def tokenize(text, word_index):
    ls = nltk.word_tokenize(text)
    return [word_index[word] if word in word_index else 1 for word in ls]

X_train = [tokenize(text, word_index) for text in train_dataset['text']]
X_val = [tokenize(text, word_index) for text in validation_dataset['text']]
X_test = [tokenize(text, word_index) for text in test_dataset['text']]
max_length = max(len(seq) for seq in X_train)

In [37]:
X_train = pad_sequences(X_train, maxlen=max_length, padding='post', truncating='post')
X_val = pad_sequences(X_val, maxlen=max_length, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=max_length, padding='post', truncating='post')

In [38]:
y_train = np.array(train_dataset['label'])
y_val = np.array(validation_dataset['label'])
y_test = np.array(test_dataset['label'])

In [39]:
def train_model(optimizer, epochs, batch_size, lr):
    tf.random.set_seed(0)
    np.random.seed(0)
    random.seed(0)
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    )
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath="model_oov.keras", 
        monitor='val_accuracy',            
        save_best_only=True,           
        mode='max',                 
        save_weights_only=False,       
        verbose=1
    )
    model = Sequential([
        Embedding(input_dim=vocab_size,
                  output_dim=embedding_dim,
                  weights=[embedding_matrix],
                  trainable=False),  # Embedding layer is frozen
        SimpleRNN(16, return_sequences=False),
        Dense(1, activation='sigmoid')
    ])
    if optimizer == 'adam': optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'sgd': optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    elif optimizer == 'rmsprop': optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    else: optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[checkpoint_callback, early_stopping]
    )
    return model, history

In [40]:
train_model("adagrad", 100, 64, 0.01)

Epoch 1/100
124/134 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5135 - loss: 0.6945
Epoch 1: val_accuracy improved from -inf to 0.52439, saving model to model_oov.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.5132 - loss: 0.6945 - val_accuracy: 0.5244 - val_loss: 0.6933
Epoch 2/100
130/134 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5322 - loss: 0.6906
Epoch 2: val_accuracy did not improve from 0.52439
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5322 - loss: 0.6906 - val_accuracy: 0.5206 - val_loss: 0.6940
Epoch 3/100
130/134 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5592 - loss: 0.6857
Epoch 3: val_accuracy improved from 0.52439 to 0.53377, saving model to model_oov.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5589 - loss: 0.6858 - val_accuracy: 0.5338 - val_loss: 0.6915
Epoch 4/100
129/134 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5767 - loss: 0.6793
Epoch 4: val_accuracy improved from 0.53377 to 0.55253, saving model to mode

(<Sequential name=sequential_71, built=True>,
 <keras.src.callbacks.history.History at 0x242b47b0fe0>)

In [41]:
best_model = tf.keras.models.load_model("model_oov.keras")
accuracy = best_model.evaluate(X_test, y_test)
print("Test accuracy:", accuracy[1])

34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6786 - loss: 0.5732
Test accuracy: 0.6763602495193481


#### 3. Keeping the above two adjustments, replace your simple RNN model in Part 2 with a biLSTM model and a biGRU model, incorporating recurrent computations in both directions and stacking multiple layers if possible

BiLSTM

In [42]:
from tensorflow.keras.layers import Bidirectional, LSTM
def train_model(optimizer, epochs, batch_size, lr):
    tf.random.set_seed(0)
    np.random.seed(0)
    random.seed(0)
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=3,
        restore_best_weights=True
    )
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath="model_combined.keras", 
        monitor='val_accuracy',            
        save_best_only=True,           
        mode='max',                 
        save_weights_only=False,       
        verbose=1
    )
    model = Sequential([
        Embedding(input_dim=vocab_size,
                  output_dim=embedding_dim,
                  weights=[embedding_matrix],
                  trainable=True),  # Embedding layer is frozen
        Bidirectional(LSTM(16, return_sequences=False)),
        Dense(1, activation='sigmoid')
    ])
    if optimizer == 'adam': optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'sgd': optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    elif optimizer == 'rmsprop': optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    else: optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[checkpoint_callback, early_stopping]
    )
    return model, history

In [43]:
train_model("adagrad", 100, 64, 0.01)

Epoch 1/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step - accuracy: 0.5296 - loss: 0.6917
Epoch 1: val_accuracy improved from -inf to 0.57599, saving model to model_combined.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 21s 134ms/step - accuracy: 0.5297 - loss: 0.6916 - val_accuracy: 0.5760 - val_loss: 0.6849
Epoch 2/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - accuracy: 0.5714 - loss: 0.6818
Epoch 2: val_accuracy improved from 0.57599 to 0.60131, saving model to model_combined.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 16s 122ms/step - accuracy: 0.5715 - loss: 0.6818 - val_accuracy: 0.6013 - val_loss: 0.6755
Epoch 3/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step - accuracy: 0.6036 - loss: 0.6692
Epoch 3: val_accuracy improved from 0.60131 to 0.63415, saving model to model_combined.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 16s 118ms/step - accuracy: 0.6036 - loss: 0.6692 - val_accuracy: 0.6341 - val_loss: 0.6607
Epoch 4/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - accuracy: 0.6412 - loss: 0.6492
Epoch 4:

(<Sequential name=sequential_72, built=True>,
 <keras.src.callbacks.history.History at 0x2419d7d1f40>)

In [44]:
best_model = tf.keras.models.load_model("model_combined.keras")
accuracy = best_model.evaluate(X_test, y_test)
print("Test accuracy:", accuracy[1])

34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6495 - loss: 0.6274 
Test accuracy: 0.7260788083076477


BiGRU

In [45]:
from tensorflow.keras.layers import Bidirectional, GRU
def train_model(optimizer, epochs, batch_size, lr):
    tf.random.set_seed(0)
    np.random.seed(0)
    random.seed(0)
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=3,
        restore_best_weights=True
    )
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath="model_combined.keras", 
        monitor='val_accuracy',            
        save_best_only=True,           
        mode='max',                 
        save_weights_only=False,       
        verbose=1
    )
    model = Sequential([
        Embedding(input_dim=vocab_size,
                  output_dim=embedding_dim,
                  weights=[embedding_matrix],
                  trainable=True),  # Embedding layer is frozen
        Bidirectional(GRU(16, return_sequences=False)),
        Dense(1, activation='sigmoid')
    ])
    if optimizer == 'adam': optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'sgd': optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    elif optimizer == 'rmsprop': optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    else: optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[checkpoint_callback, early_stopping]
    )
    return model, history

In [46]:
train_model("adagrad", 100, 64, 0.01)

Epoch 1/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step - accuracy: 0.5007 - loss: 0.7018
Epoch 1: val_accuracy improved from -inf to 0.50750, saving model to model_combined.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 20s 127ms/step - accuracy: 0.5007 - loss: 0.7017 - val_accuracy: 0.5075 - val_loss: 0.6927
Epoch 2/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - accuracy: 0.5439 - loss: 0.6876
Epoch 2: val_accuracy improved from 0.50750 to 0.55159, saving model to model_combined.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 15s 115ms/step - accuracy: 0.5440 - loss: 0.6876 - val_accuracy: 0.5516 - val_loss: 0.6857
Epoch 3/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step - accuracy: 0.5801 - loss: 0.6800
Epoch 3: val_accuracy improved from 0.55159 to 0.55910, saving model to model_combined.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 16s 116ms/step - accuracy: 0.5801 - loss: 0.6800 - val_accuracy: 0.5591 - val_loss: 0.6803
Epoch 4/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - accuracy: 0.5972 - loss: 0.6738
Epoch 4:

(<Sequential name=sequential_73, built=True>,
 <keras.src.callbacks.history.History at 0x24293788fe0>)

In [48]:
best_model = tf.keras.models.load_model("model_combined.keras")
accuracy = best_model.evaluate(X_test, y_test)
print("Test accuracy:", accuracy[1])

34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7120 - loss: 0.5646
Test accuracy: 0.7532833218574524


#### 4. Keeping the above two adjustments, replace your simple RNN model in Part 2 with a Convolutional Neural Network (CNN) to produce sentence representations and perform sentiment classification

In [49]:
from tensorflow.keras.layers import Convolution1D, Flatten

def train_model(optimizer, epochs, batch_size, lr):
    tf.random.set_seed(0)
    np.random.seed(0)
    random.seed(0)
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=3,
        restore_best_weights=True
    )
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath="model_combined.keras", 
        monitor='val_accuracy',            
        save_best_only=True,           
        mode='max',                 
        save_weights_only=False,       
        verbose=1
    )
    model = Sequential([
        Embedding(input_dim=vocab_size,
                  output_dim=embedding_dim,
                  weights=[embedding_matrix],
                  trainable=True),  # Embedding layer is trainable
        Convolution1D(16, kernel_size=3, activation='relu'),
        Flatten(),
        Dense(1, activation='sigmoid')
    ])
    if optimizer == 'adam': optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'sgd': optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    elif optimizer == 'rmsprop': optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    else: optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[checkpoint_callback, early_stopping]
    )
    return model, history

In [52]:
model, history = train_model("adagrad", 100, 64, 0.01)

Epoch 1/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 0.4938 - loss: 0.7037
Epoch 1: val_accuracy improved from -inf to 0.53846, saving model to model_combined.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 19s 133ms/step - accuracy: 0.4939 - loss: 0.7036 - val_accuracy: 0.5385 - val_loss: 0.6903
Epoch 2/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 0.5456 - loss: 0.6871
Epoch 2: val_accuracy improved from 0.53846 to 0.58161, saving model to model_combined.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 18s 131ms/step - accuracy: 0.5457 - loss: 0.6870 - val_accuracy: 0.5816 - val_loss: 0.6789
Epoch 3/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step - accuracy: 0.5965 - loss: 0.6690
Epoch 3: val_accuracy improved from 0.58161 to 0.60694, saving model to model_combined.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 18s 131ms/step - accuracy: 0.5966 - loss: 0.6690 - val_accuracy: 0.6069 - val_loss: 0.6650
Epoch 4/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - accuracy: 0.6333 - loss: 0.6457
Epoch 4:

In [53]:
best_model = tf.keras.models.load_model("model_combined.keras")
accuracy = best_model.evaluate(X_test, y_test)
print("Test accuracy:", accuracy[1])

34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6092 - loss: 0.7034
Test accuracy: 0.7101313471794128
